# MLflow Test Notebook

This notebook runs key components from the MLflow evaluation project.

**Prerequisites:**
- MLflow server running on http://127.0.0.1:5000
- Required packages installed (mlflow, scikit-learn, pandas, seaborn, matplotlib)

**Run this notebook cell by cell to test the complete MLflow workflow.**

In [ ]:
# Cell 1: Setup and Imports
import warnings
warnings.filterwarnings('ignore')

import os
import random
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris, load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.models.signature import infer_signature
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime

# Set tracking URI
TRACKING_URI = 'http://127.0.0.1:5000'
mlflow.set_tracking_uri(TRACKING_URI)
client = MlflowClient()

print("MLflow tracking URI:", mlflow.get_tracking_uri())

In [ ]:
# Cell 2: Load Data
iris = load_iris(as_frame=True)
X_iris = iris.data
y_iris = iris.target

wine = load_wine(as_frame=True)
X_wine = wine.data
y_wine = wine.target

# Titanic data
try:
    titanic = sns.load_dataset('titanic')
    titanic = titanic.dropna(subset=['survived'])
    titanic = titanic.select_dtypes(include=['number']).fillna(0)
    X_titanic = titanic.drop(columns=['survived'])
    y_titanic = titanic['survived']
    print("Titanic dataset loaded successfully")
except Exception as e:
    print(f"Titanic dataset not available: {e}")
    X_titanic = None
    y_titanic = None

print("Data loading completed")

In [ ]:
# Cell 3: Define Training Function
def train_and_log_baselines(X, y, dataset_name='dataset', seed=42):
    experiment_name = f"{dataset_name} experiment"
    try:
        mlflow.set_experiment(experiment_name)
    except Exception as e:
        print(f'Warning: could not set experiment "{experiment_name}": {e}')

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

    models = {
        'random_forest': RandomForestClassifier(random_state=seed),
        'logistic_regression': LogisticRegression(max_iter=500)
    }

    for name, model in models.items():
        with mlflow.start_run(nested=True):
            mlflow.log_param('model', name)
            mlflow.log_param('dataset', dataset_name)
            mlflow.log_param('seed', seed)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            prec = precision_score(y_test, y_pred, average='weighted', zero_division=0)
            rec = recall_score(y_test, y_pred, average='weighted', zero_division=0)
            f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
            mlflow.log_metrics({'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1})

            try:
                input_example = X_train.head(5)
                sample_preds = model.predict(input_example)
                signature = infer_signature(input_example, sample_preds)
                mlflow.sklearn.log_model(model, artifact_path='./artifacts/', signature=signature, input_example=input_example)
            except Exception as e:
                print(f"Model logging failed: {e}")
                try:
                    mlflow.sklearn.log_model(model, artifact_path='./artifacts/')
                except Exception:
                    pass

print("Training function defined")

In [ ]:
# Cell 4: Train Baseline Models
# Generate seeds
seeds = [random.randint(0, 10000) for _ in range(3)]
print(f'Using seeds for testing: {seeds}')

# Train baselines
for seed in seeds:
    train_and_log_baselines(X_iris, y_iris, dataset_name='iris', seed=seed)
    if X_titanic is not None:
        train_and_log_baselines(X_titanic, y_titanic, dataset_name='titanic', seed=seed)
    train_and_log_baselines(X_wine, y_wine, dataset_name='wine', seed=seed)

print("Baseline training completed")

In [ ]:
# Cell 5: Model Registry - Train and Register Versions
mlflow.set_experiment("Model Registry Demo")
model_name = "IrisClassifier"

model_configs = [
    {"n_estimators": 10, "max_depth": 3},
    {"n_estimators": 50, "max_depth": 5},
    {"n_estimators": 100, "max_depth": 10}
]

X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris, test_size=0.2, random_state=42)

for i, config in enumerate(model_configs, 1):
    with mlflow.start_run(run_name=f"version_{i}"):
        model = RandomForestClassifier(n_estimators=config['n_estimators'], max_depth=config['max_depth'], random_state=42)
        model.fit(X_train, y_train)
        mlflow.log_params(config)
        
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)
        
        model_info = mlflow.sklearn.log_model(model, artifact_path="model", registered_model_name=model_name)
        print(f"Registered {model_name} version {i} - Accuracy: {accuracy:.4f}")

print("Model versions registered")

In [ ]:
# Cell 6: Stage Transitions
# Transition version 1 to Staging
client.transition_model_version_stage(name=model_name, version=1, stage="Staging")
print("Version 1 moved to Staging")

# Transition version 2 to Staging (archive v1)
client.transition_model_version_stage(name=model_name, version=2, stage="Staging", archive_existing_versions=True)
print("Version 2 moved to Staging (Version 1 archived)")

# Promote version 3 to Production
client.transition_model_version_stage(name=model_name, version=3, stage="Production")
print("Version 3 moved to Production")

print("Stage transitions completed")

In [ ]:
# Cell 7: Create Directories and Load Production Model
# Create directories
os.makedirs('model_artifacts', exist_ok=True)
os.makedirs('deployment', exist_ok=True)

# Load production model
model_uri = f"models:/{model_name}/Production"
prod_model = mlflow.sklearn.load_model(model_uri)

y_pred = prod_model.predict(X_test)
y_pred_proba = prod_model.predict_proba(X_test)

print("Production model loaded successfully")

In [ ]:
# Cell 8: Generate and Log Artifacts - Confusion Matrix
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.title('Confusion Matrix - Production Model')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig('model_artifacts/confusion_matrix.png', bbox_inches='tight', dpi=150)
plt.close()
mlflow.log_artifact('model_artifacts/confusion_matrix.png')

print("Confusion matrix saved and logged")

In [ ]:
# Cell 9: Generate and Log Artifacts - Feature Importance
# Feature Importance
feature_importance = pd.DataFrame({
    'feature': iris.feature_names,
    'importance': prod_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['importance'])
plt.xlabel('Importance')
plt.title('Feature Importance - Production Model')
plt.gca().invert_yaxis()
plt.savefig('model_artifacts/feature_importance.png', bbox_inches='tight', dpi=150)
plt.close()
mlflow.log_artifact('model_artifacts/feature_importance.png')

print("Feature importance plot saved and logged")

In [ ]:
# Cell 10: Generate and Log Artifacts - Classification Report and Metadata
# Classification Report
report = classification_report(y_test, y_pred, target_names=iris.target_names, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_csv('model_artifacts/classification_report.csv')
mlflow.log_artifact('model_artifacts/classification_report.csv')

# Model Metadata
metadata = {
    "model_name": model_name,
    "version": "3 (Production)",
    "training_date": datetime.now().isoformat(),
    "framework": "scikit-learn",
    "algorithm": "RandomForestClassifier",
    "metrics": {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, average='weighted'),
        "recall": recall_score(y_test, y_pred, average='weighted'),
        "f1_score": f1_score(y_test, y_pred, average='weighted')
    },
    "feature_names": iris.feature_names,
    "target_names": iris.target_names.tolist(),
    "n_samples_train": len(X_train),
    "n_samples_test": len(X_test)
}
with open('model_artifacts/model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)
mlflow.log_artifact('model_artifacts/model_metadata.json')

print("Classification report and metadata saved and logged")

In [ ]:
# Cell 11: Batch Inference Test
test_samples = pd.DataFrame({
    'sepal length (cm)': [5.1, 6.7, 4.9],
    'sepal width (cm)': [3.5, 3.0, 2.5],
    'petal length (cm)': [1.4, 5.2, 4.5],
    'petal width (cm)': [0.2, 2.3, 1.7]
})

predictions = prod_model.predict(test_samples)
probabilities = prod_model.predict_proba(test_samples)

print("Batch Inference Results:")
print("-" * 50)
for i, (pred, probs) in enumerate(zip(predictions, probabilities)):
    pred_name = iris.target_names[pred]
    confidence = probs[pred] * 100
    print(f"Sample {i+1}: {pred_name} (confidence: {confidence:.1f}%)")

print("\nBatch inference test completed!")

In [ ]:
# Cell 12: Create Serving Scripts
# Serving instructions
serving_instructions = """
# Model Serving Instructions
# 1. Start MLflow server: mlflow server --host 127.0.0.1 --port 5000
# 2. Serve model: mlflow models serve -m models:/IrisClassifier/Production -p 5001 --no-conda
# 3. Test API: curl -X POST -H "Content-Type: application/json" \
#      --data '{"dataframe_split": {"columns": ["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"], "data": [[5.1, 3.5, 1.4, 0.2]]}}' \
#      http://127.0.0.1:5001/invocations
"""

with open('deployment/serving_instructions.txt', 'w') as f:
    f.write(serving_instructions)

# Batch inference script
inference_script = """import mlflow
import pandas as pd

# Load production model
model = mlflow.sklearn.load_model("models:/IrisClassifier/Production")

# Test data
data = pd.DataFrame({
    'sepal length (cm)': [5.1, 6.2, 5.9],
    'sepal width (cm)': [3.5, 3.4, 3.0],
    'petal length (cm)': [1.4, 5.4, 5.1],
    'petal width (cm)': [0.2, 2.3, 1.8]
})

# Make predictions
predictions = model.predict(data)
probabilities = model.predict_proba(data)

# Display results
target_names = ['setosa', 'versicolor', 'virginica']
for i, (pred, probs) in enumerate(zip(predictions, probabilities)):
    pred_name = target_names[pred]
    confidence = probs[pred] * 100
    print(f"Sample {i+1}: {pred_name} (confidence: {confidence:.1f}%)")
"""

with open('deployment/batch_inference.py', 'w') as f:
    f.write(inference_script)

print("Serving scripts created in deployment/ directory")

## Summary

This notebook has successfully:
1. ✅ Trained baseline models and logged experiments
2. ✅ Registered multiple model versions in MLflow Model Registry
3. ✅ Performed stage transitions (Staging → Production)
4. ✅ Generated and logged artifacts (plots, reports, metadata)
5. ✅ Tested batch inference with the production model
6. ✅ Created deployment scripts for model serving

### Next Steps:
- Visit http://127.0.0.1:5000 to explore the MLflow UI
- Run the serving script: `mlflow models serve -m models:/IrisClassifier/Production -p 5001 --no-conda`
- Test the REST API with curl commands
- Check the generated artifacts in the `model_artifacts/` directory